<a href="https://colab.research.google.com/github/danielpiyo/NeuralStyleTransfer/blob/main/RestylingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
import torch
import torch.nn as nn
import torch.optim as optimizer
from PIL import Image
import torchvision.transforms as transforms
import torchvision.models as models
from torchvision.utils import save_image
from torch.optim import Adam


In [57]:
model = models.vgg19(pretrained=True).features
# print(model)

In [58]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [66]:
style_image_path = "/content/drive/MyDrive/modigliani_style/modigliani_image.jpg"
input_image_path = "/content/drive/MyDrive/modigliani_style/input_image.jpg"

# Define the paths to the input and output images
output_image_path = '/content/drive/MyDrive/modigliani_style/output_modigliani_style_image.jpg'

In [67]:
class VGG(nn.Module):
  def __init__(self) -> None:
      super(VGG, self).__init__()
      self.chosen_features = ['0', '5', '10', '19', '28']
      self.model = models.vgg19(pretrained = True).features[:29]
  def forward(self, x):
    features = []
    for llayer_num, layer in enumerate(self.model):
      x = layer(x)
      if str(llayer_num) in self.chosen_features:
        features.append(x)
    return features


In [68]:
device = torch.device("cuda" if torch.cuda.is_available else "cpu")
# device = torch.device("cpu")
image_size = 356
loader = transforms.Compose([transforms.Resize((image_size, image_size)), transforms.ToTensor()])


In [69]:
def load_image(image_name):
  image = Image.open(image_name)
  image = loader(image).unsqueeze(0)
  return image.to(device)

In [70]:
original_image = load_image(input_image_path)
style_image = load_image(style_image_path)
# print(original_image)
# model = VGG.to(device).eval()
model = VGG().to(device).eval()
# generated = torch.randn(original_image, device = device, requires_grad=True)
# generated_image = original_image.clone().requires_grad(True)
# generated_image = torch.Tensor(original_image).clone().requires_grad_(True)
generated_image = torch.Tensor(original_image[:,:3,:,:]).clone().requires_grad_(True)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [71]:
# Hyperparameters
total_steps = 6000
learning_rate = 0.01
alpha = 1
beta = 0.01
# optimizer = optimizer.Adam([generated_image],lr=learning_rate )
optimizer = Adam([generated_image], lr=learning_rate)


In [72]:
from torch.optim.optimizer import Optimizer

original_image = original_image[:, :3, :, :]
for step in range(total_steps):
  generated_features = model(generated_image)
  original_image_features = model(original_image)
  style_features = model(style_image)

  style_loss = original_loss = 0

  for gen_feature, orig_feature, style_feature in zip(
      generated_features, original_image_features, style_features):
    batch_size, channel, height, width = gen_feature.shape
    original_loss += torch.mean((gen_feature - orig_feature) **2)

    # compute gram Matrix
    G = gen_feature.view(channel, height*width).mm(gen_feature.view(channel, height*width).t())
    A = style_feature.view(channel, height*width).mm(style_feature.view(channel, height*width).t())

    style_loss += torch.mean((G-A)**2)

  total_loss = alpha*original_loss + beta*style_loss
  optimizer.zero_grad()
  total_loss.backward()
  optimizer.step()

  if step %200 == 0:
    print('Iteration %d completed' % step)
    print(total_loss)
    save_image(generated_image, output_image_path)


Iteration 0 completed
tensor(340352.4375, device='cuda:0', grad_fn=<AddBackward0>)
Iteration 200 completed
tensor(12991.7480, device='cuda:0', grad_fn=<AddBackward0>)
Iteration 400 completed
tensor(8076.8159, device='cuda:0', grad_fn=<AddBackward0>)
Iteration 600 completed
tensor(5576.0225, device='cuda:0', grad_fn=<AddBackward0>)
Iteration 800 completed
tensor(4129.9985, device='cuda:0', grad_fn=<AddBackward0>)
Iteration 1000 completed
tensor(3542.6001, device='cuda:0', grad_fn=<AddBackward0>)
Iteration 1200 completed
tensor(2902.7661, device='cuda:0', grad_fn=<AddBackward0>)
Iteration 1400 completed
tensor(2524.3901, device='cuda:0', grad_fn=<AddBackward0>)
Iteration 1600 completed
tensor(2259.7920, device='cuda:0', grad_fn=<AddBackward0>)
Iteration 1800 completed
tensor(2056.9226, device='cuda:0', grad_fn=<AddBackward0>)
Iteration 2000 completed
tensor(1905.9907, device='cuda:0', grad_fn=<AddBackward0>)
Iteration 2200 completed
tensor(1783.6993, device='cuda:0', grad_fn=<AddBackward